In [1]:
from pandas import read_csv, DataFrame
import pandas as pd
import os
import re #Both patterns and strings to be searched can be Unicode strings as well as 8-bit strings.
import math
import cobra
import cobra.test
from __future__ import print_function
from os.path import join

In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
relative_directory = os.getcwd()
print(relative_directory)

data_dir = "/Users/lizrad/Documents/Vibrio_folder"
print("files found: ")
print(", ".join(i for i in os.listdir(data_dir) if i.startswith("GCF")))
model=cobra.io.read_legacy_sbml(join(data_dir, "GCF_001456255.1_rast_metabolic_model.xml"))
#model=cobra.io.read_legacy_sbml("C:\\Users\Asus\Documents\Vibrio_project_literature\GCF_001456255.1_rast_metabolic_model.SBML\GCF_001456255.1_rast_metabolic_model.xml")
model

/Users/lizrad/Dev/iVnat
files found: 
GCF_001456255.1_rast_metabolic_model.xml


Name,GCF_001456255_1_rast_metabolic_model
Memory address,0x0111b9c5c0
Number of metabolites,1842
Number of reactions,1906
Objective expression,-1.0*bio1_reverse_b18f7 + 1.0*bio1
Compartments,"Cytosol_0, Extracellular_0"


# Metabolites

## Read chem_prop

In [ ]:
chem_prop = read_csv("/Users/lizrad/Documents/Vibrio_folder/chem_prop.tsv" , sep='\t' , skiprows=365)
chem_prop

In [ ]:

chem_prop_reordered = chem_prop.set_index("#MNX_ID")
chem_prop_reordered
chem_prop_dict = chem_prop_reordered.to_dict('index')
chem_prop_dict







### Read chem_xref

In [ ]:
chem_xref = read_csv("/Users/lizrad/Documents/Vibrio_folder/chem_xref3.tsv" , sep='\t' , low_memory=False)
chem_xref


In [ ]:
df_2 = chem_xref.loc[:,'XREF':'MNX_ID']
df_2
groups = df_2.groupby(['MNX_ID','XREF'])
df_2 = groups.apply(lambda x:list(x['XREF_ID']))
df_2
df_2 =df_2.unstack('XREF')
df_2
chem_xref_MNX_2_dict = df_2.to_dict('index')
chem_xref_MNX_2_dict




### SEED IDs to dict key

In [ ]:

#499091 530559
chem_xref_MNX_seed = read_csv("/Users/lizrad/Documents/Vibrio_folder/chem_xref2.tsv" , sep='\t', low_memory=False)
df_seed = chem_xref.loc[499088:530556,'XREF':'MNX_ID']
pivoted_seed = df_seed.pivot_table(index='XREF',columns='XREF_ID',values='MNX_ID',aggfunc = lambda x: x)
pivoted_seedd=pivoted_seed.drop(['metanetx'])
chem_xref_MNX_seed_dict = pivoted_seedd.to_dict('index')
chem_xref_MNX_seed_dict2 = chem_xref_MNX_seed_dict['seed']


In [ ]:
pivoted_seed.drop(['metanetx'])

In [ ]:
chem_xref_MNX_seed_dict2

### Check how many Metabolites can be found in the SEED database and how many cannot be found at all. 

In [ ]:
print ('This is the total amount of metabolites in the model: %i' % (len(model.metabolites)))

print ('We focus on mapping those %i metabolites in the model that have an MNX ID' % (len([met for met in model.metabolites if met.id.startswith('cpd')])))

### How many SEED metabolites can we find?

In [ ]:
for met in model.metabolites:
    id_split=met.id.split("_")
    print(id_split[0])
    

In [ ]:
  met_clean, compartment=model.metabolites.cpd00001_e0.id.split("_")
    

In [ ]:
not_in_chem_xref = []
in_chem_xref = []




for met in model.metabolites:
    split=met.id.split("_")
    met_clean=split[0]
    print(met_clean)
    if not met_clean == '':
        if not met_clean in chem_xref_MNX_seed_dict2.keys():
            not_in_chem_xref.append(met)
        else:
            in_chem_xref.append(met)

print ('%i metabolites with SEED ID cannot be found in the MNX database' % (len(not_in_chem_xref)))




### Seed ids to annotation field

In [ ]:
len(in_chem_xref)

In [ ]:
for met in model.metabolites:
    split, compartment=met.id.split("_")
    met.annotation['seed.compound']=split
    print(met.annotation)
    compartment



### MNX ids to annotation field

In [ ]:
for met in model.metabolites:
    print(met.id)

In [ ]:
for met in model.metabolites:
    data=''
    if 'seed.compound' in met.annotation:
        data=met.annotation['seed.compound'] 
        
        for xref in chem_xref_MNX_seed_dict2.keys():
            if xref==data:
                met.annotation['metanetx.chemical']=chem_xref_MNX_seed_dict2[xref]
                print(met.annotation)
              
            
                

In [ ]:

#for met in model.metabolites:
#    if met.id.startswith('MNXM'):
#        met.id=met.annotation['seed.compound']
 #       print(met.id)
    
        

In [ ]:
dupl = {}
for met in model.metabolites:
    if met.annotation["metanetx.chemical"] not in dupl.keys():
        dupl.setdefault(met.annotation["metanetx.chemical"],[met.id])
    else:
        dupl[met.annotation["metanetx.chemical"]].append(met.id)

In [ ]:
for ls in dupl.values():
    if len(ls) > 1:
        print(ls)

In [ ]:
for ls in dupl.values():
    if len(ls) > 1:
        #print(ls)
        x=[]
        y=[]
        for members in ls: 
            main_id, comp =members.split("_")
            x.append(main_id)
            y.append(comp)
        if not x[0]==x[1]:
            print(x,y)
            reaction_involved=[]
            for pair in zip(x,y):
                react_number=len(model.metabolites.get_by_id(pair[0]+"_"+pair[1]).reactions)
                reaction_involved.append(react_number)
                print (react_number)
            
        #if not y[0] == y[1]: 
        #   print("different compartment same number", x,y)

In [ ]:
for ls in dupl.values():
    if len(ls) > 1:
        #print(ls)
        x=[]
        y=[]
        for members in ls: 
            main_id, comp =members.split("_")
            x.append(main_id)
            y.append(comp)
        if not x[0]==x[1]:
            print(x,y)
            for pair in zip(x,y):
                print(len(model.metabolites.get_by_id(pair[0]+"_"+pair[1]).reactions))
        #if x[0]==x[1] and y[0]==y[1]:
         #   print(x,y)
        #if not y[0] == y[1]:
         #   print("different compartment same number", x,y)

In [ ]:
for met in model.metabolites:
    split, compartment=met.id.split("_")
    #if met.id.startswith('cpd'):
    met.id
    if met.id in ["cpd00261_c0", "cpd06227_c0", "cpd03572_c0", "cpd02446_c0","cpd02572_c0", "cpd01466_c0" ]:
        pass
    else:
        met.id=met.annotation['metanetx.chemical']+ '_' + compartment
        model.repair()
    print(met.id)
    



In [ ]:
model.metabolites.query("H_plus_", attribute="name")




model.metabolites.cpd00001_c0.formula

In [ ]:
copy_met = model.metabolites.cpd00002_c0

In [ ]:
copy_met.annotation["seed.compound"] = "cpd00002"

In [ ]:
copy_met.id = ""

In [ ]:
# Quick check-up to compare conditions before and after:
print (len(model.metabolites))
print (len([met for met in model.metabolites if met.formula == 'c0']))  # ??? none of them has a formula
print (len([met for met in model.metabolites if met.formula == '' and met.id.startswith('cpd')]))

print (len([met for met in model.metabolites if not met.charge]))
print (len([met for met in model.metabolites if not met.charge and met.id.startswith('cpd')]))


for met in model.metabolites:
    print(met.id.startswith('cpd'))

In [ ]:
# No KEGG ID or the KEGG IDs don't match.
no_match_via_kegg = []
# Matched
jackpot = []

for met in in_chem_xref:
    if 'seed' in met.notes and met.notes['seed'] != '':
        if met.notes['seed'][0] in chem_xref_MNX_seed_dict:
            updated_mnx_id = chem_xref_MNX_2_dict[met.notes['seed'][0]]
            temp_id = met.id.split('_')
            met.id = updated_mnx_id +'_'+ temp_id[1]
            model.repair()
            met.notes['MXNREF'] = [updated_mnx_id ]
            jackpot.append(met)
        else:
            no_match_via_kegg.append(met)
    else:
        no_match_via_kegg.append(met)
        

print(len(jackpot))
print(len(no_match_via_kegg))

In [70]:
from cobra.io import write_sbml_model

In [ ]:
write_sbml_model(model, "/Users/lizrad/Dev/iVnat/iVnat.xml")

###  Mapping from seed to MNX ??

In [ ]:
MNX2Bigg_remapped_2 = 0 
other_XRef_2 = []
x = float('NaN')

for met in not_in_chem_xref:  
    met_clean = ''.join(re.findall('(cpd\d*)',met.id)) #not_in_chem_prop2 is frozenset and type(chem_xref_MNX_seed_dict[met_clean]['MNX_ID']) != type(x)
    if met_clean in chem_xref_MNX_seed_dict.keys()  and chem_xref_MNX_seed_dict[met_clean]['MNX_ID'] != []:
        bigg_id = chem_xref_seed_dict[met_clean]['MNX_ID'][0]  
        temp_id = met.id.split('_')
        print (bigg_id)
        met.id = str(bigg_id) +'_'+ temp_id[1]
        model.repair()   #how does it work
        MNX2Bigg_remapped_2 += 1
    else:
        other_XRef_2.append(met)
        
print ('%i mets were successfully remapped from MNX to BIGG based on ver.2 MNX Chem XRef' % (MNX2Bigg_remapped_2))

print ('%i metabolites with MNX ID cannot be found or can only be mapped to a different ID in ver. 2 ' % (len(other_XRef_2)))

### Reading the reac_prop

In [ ]:
reac_prop = read_csv("C:\\Users\Asus\iVnat\reac_prop.tsv" , sep='\t' , skiprows=365)
reac_prop_reordered = chem_prop.set_index("#MNX_ID")
reac_prop_dict = chem_prop_reordered.to_dict('index')

### Reading the reac_xref

In [20]:
reac_xref = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref2.tsv" , sep='\t')
reac_xref

,XREF,XREF_ID,MNX_ID
0,BIOMASS_EXT,MNXR94667,NaN
1,seed,rxn12985,MNXR94667
2,metanetx,MNXR94667,MNXR94667
3,metanetx,MNXR01,MNXR01
4,bigg,10FTHF5GLUtl,MNXR94668
5,metanetx,MNXR94668,MNXR94668
6,bigg,10FTHF5GLUtm,MNXR94668
7,bigg,10FTHF6GLUtl,MNXR94669
8,metanetx,MNXR94669,MNXR94669
9,bigg,10FTHF6GLUtm,MNXR94669


### Number of seed reactions in the model

In [40]:
print ('This is the total amount of reactions in the model: %i' % (len(model.reactions)))

print ('We focus on mapping those %i reactions in the model that have an SEED ID' % (len([met for met in model.reactions if met.id.startswith('rxn') ])))

for met in model.reactions:
    if not met.id.startswith('rxn'):
        print(met.id)

This is the total amount of reactions in the model: 1906
We focus on mapping those 1776 reactions in the model that have an SEED ID
bio1
EX_cpd00067_e0
EX_cpd00182_e0
EX_cpd00106_e0
EX_cpd00075_e0
EX_cpd00064_e0
EX_cpd08636_c0
EX_cpd00036_e0
EX_cpd11576_e0
EX_cpd04097_e0
EX_cpd11585_e0
EX_cpd00412_e0
EX_cpd11582_e0
EX_cpd00012_e0
EX_cpd11586_e0
EX_cpd01030_e0
EX_cpd00129_e0
EX_cpd00971_e0
EX_cpd01012_e0
EX_cpd00635_e0
EX_cpd00246_e0
EX_cpd00276_e0
EX_cpd00082_e0
EX_cpd00001_e0
EX_cpd00023_e0
EX_cpd00367_e0
EX_cpd00058_e0
EX_cpd00080_e0
EX_cpd00107_e0
EX_cpd00184_e0
EX_cpd00314_e0
EX_cpd01017_e0
EX_cpd00797_e0
EX_cpd00013_e0
EX_cpd00051_e0
EX_cpd10515_e0
EX_cpd00308_e0
EX_cpd00179_e0
EX_cpd00100_e0
EX_cpd11591_e0
EX_cpd00009_e0
EX_cpd00060_e0
EX_cpd00438_e0
EX_cpd00034_e0
EX_cpd00249_e0
EX_cpd00214_e0
EX_cpd00119_e0
EX_cpd11593_e0
EX_cpd00654_e0
EX_cpd00076_e0
EX_cpd00322_e0
EX_cpd01171_e0
EX_cpd00210_e0
EX_cpd15302_c0
EX_cpd15605_e0
EX_cpd00264_e0
EX_cpd00054_e0
EX_cpd00644_e0
EX_cpd00

In [21]:
seed_reactions = [rxn for rxn in model.reactions if rxn.id.startswith('rxn')]

print (len(seed_reactions))
print (len(model.reactions))

1776
1906


In [ ]:
#skip
rxn_mnx2bigg = []
rxn_nobiggid = []

for rxn in [rxn for rxn in cameo_model.reactions if rxn.id.startswith('rxn')]:
    if 'BIGG' in rxn.notes:
        reaction_id = rxn.annotation['BIGG'][0].split(', ')
        tag_split = rxn.id.split('_')
        rxn.id = str(reaction_id[0]) +'_'+ tag_split[1]
        rxn_mnx2bigg.append(reaction_id)
        cameo_model.repair()
    else:
        rxn_nobiggid.append(rxn)
        
print len(rxn_mnx2bigg)
print len(rxn_nobiggid)

### Formatting and creating SEED MNX dictionary

In [22]:

df_1 = reac_xref.loc[:,'XREF':'MNX_ID']
groups = df_1.groupby(['MNX_ID','XREF'])
df_1 = groups.apply(lambda x:list(x['XREF_ID']))
df_1 =df_1.unstack('XREF') 
reac_xref_MNX_1_dict = df_1.to_dict('index')
reac_xref_MNX_1_dict

{'MNXR01': {'metanetx': ['MNXR01'],
  'bigg': nan,
  'deprecated': nan,
  'kegg': nan,
  'seed': nan,
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100000': {'metanetx': ['MNXR100000'],
  'bigg': ['GALNACT5g'],
  'deprecated': ['MNXR55538'],
  'kegg': nan,
  'seed': nan,
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100001': {'metanetx': ['MNXR100001'],
  'bigg': ['GALNTg', 'NAGA2ly'],
  'deprecated': ['MNXR55539', 'MNXR69102', 'MNXR82961'],
  'kegg': nan,
  'seed': nan,
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100002': {'metanetx': ['MNXR100002'],
  'bigg': ['GALO'],
  'deprecated': ['MNXR65868', 'MNXR35564'],
  'kegg': ['R01098'],
  'seed': ['rxn00814'],
  'metacyc': nan,
  'rhea': nan,
  'sabiork': nan,
  'reactome': nan},
 'MNXR100003': {'metanetx': ['MNXR100003'],
  'bigg': ['GALOR'],
  'deprecated': ['MNXR31513', 'MNXR2525'],
  'kegg': ['R01095'],
  'seed': ['rxn11552'],
  'metacyc':

In [23]:

reac_property_MNX = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref2.tsv" , sep='\t')
reac_property_MNX.set_index('MNX_ID',inplace = True)
df_3 = reac_property_MNX.fillna('MISSING')  #Fill NA/NaN values using the specified method
df_3
reac_property_MNX_dict = df_3.to_dict('index')
reac_property_MNX_dict

,XREF,XREF_ID
MNX_ID,,
NaN,BIOMASS_EXT,MNXR94667
MNXR94667,seed,rxn12985
MNXR94667,metanetx,MNXR94667
MNXR01,metanetx,MNXR01
MNXR94668,bigg,10FTHF5GLUtl
MNXR94668,metanetx,MNXR94668
MNXR94668,bigg,10FTHF5GLUtm
MNXR94669,bigg,10FTHF6GLUtl
MNXR94669,metanetx,MNXR94669


{nan: {'XREF': 'BIOMASS_EXT', 'XREF_ID': 'MNXR94667'},
 'MNXR94667': {'XREF': 'metanetx', 'XREF_ID': 'MNXR94667'},
 'MNXR01': {'XREF': 'metanetx', 'XREF_ID': 'MNXR01'},
 'MNXR94668': {'XREF': 'deprecated', 'XREF_ID': 'MNXR1'},
 'MNXR94669': {'XREF': 'deprecated', 'XREF_ID': 'MNXR2'},
 'MNXR94670': {'XREF': 'deprecated', 'XREF_ID': 'MNXR3'},
 'MNXR94671': {'XREF': 'bigg', 'XREF_ID': '10FTHFGLULLm'},
 'MNXR94672': {'XREF': 'deprecated', 'XREF_ID': 'MNXR4'},
 'MNXR94673': {'XREF': 'deprecated', 'XREF_ID': 'MNXR5'},
 'MNXR94674': {'XREF': 'deprecated', 'XREF_ID': 'MNXR6'},
 'MNXR94675': {'XREF': 'deprecated', 'XREF_ID': 'MNXR7'},
 'MNXR94676': {'XREF': 'deprecated', 'XREF_ID': 'MNXR81306'},
 'MNXR94677': {'XREF': 'deprecated', 'XREF_ID': 'MNXR9'},
 'MNXR94678': {'XREF': 'deprecated', 'XREF_ID': 'MNXR10'},
 'MNXR94679': {'XREF': 'deprecated', 'XREF_ID': 'MNXR11'},
 'MNXR94680': {'XREF': 'deprecated', 'XREF_ID': 'MNXR12'},
 'MNXR94681': {'XREF': 'deprecated', 'XREF_ID': 'MNXR13'},
 'MNXR9468

In [ ]:
#skip
rxn_ID_not_in_MetaNetX_2 = [rxn for rxn in rxn_nobiggid if rxn.id.startswith('MNXR') and ''.join(re.findall('(MNXR\d*)_',rxn.id)) not in reac_xref_MNX_2_dict.keys()]
print len(rxn_ID_not_in_MetaNetX_2)

In [28]:
# Get a mapping from KEGG to the new MNXR IDs, for those reactions that cannot be found in the MNX Database,
# but that have a KEGG ID in their annotation. 
#153644- 170637
df_2 = reac_xref.loc[153642:170637,'XREF_ID':'MNX_ID']
df_2 = df_2.set_index(['XREF_ID'])
seed_2_mnx_dict_2 = df_2.to_dict('index')
seed_2_mnx_dict_2

{'rxn00001': {'MNX_ID': 'MNXR100808'},
 'rxn00002': {'MNX_ID': 'MNXR95786'},
 'rxn00003': {'MNX_ID': 'MNXR106335'},
 'rxn00004': {'MNX_ID': 'MNXR106336'},
 'rxn00006': {'MNX_ID': 'MNXR96455'},
 'rxn00007': {'MNX_ID': 'MNXR95163'},
 'rxn00008': {'MNX_ID': 'MNXR134184'},
 'MNXR134184': {'MNX_ID': 'MNXR134184'},
 'rxn00009': {'MNX_ID': 'MNXR106338'},
 'rxn00010': {'MNX_ID': 'MNXR100303'},
 'rxn00011': {'MNX_ID': 'MNXR102428'},
 'rxn00012': {'MNX_ID': 'MNXR106339'},
 'rxn00013': {'MNX_ID': 'MNXR106765'},
 'rxn00014': {'MNX_ID': 'MNXR134185'},
 'MNXR134185': {'MNX_ID': 'MNXR134185'},
 'rxn00015': {'MNX_ID': 'MNXR100740'},
 'rxn00016': {'MNX_ID': 'MNXR95628'},
 'rxn00017': {'MNX_ID': 'MNXR106341'},
 'rxn00018': {'MNX_ID': 'MNXR103435'},
 'rxn00019': {'MNX_ID': 'MNXR102013'},
 'rxn00020': {'MNX_ID': 'MNXR126492'},
 'rxn00021': {'MNX_ID': 'MNXR106344'},
 'rxn00022': {'MNX_ID': 'MNXR101350'},
 'rxn00023': {'MNX_ID': 'MNXR134186'},
 'MNXR134186': {'MNX_ID': 'MNXR134186'},
 'rxn00024': {'MNX_ID':

In [34]:
reac_xref_MNX_seed = read_csv("/Users/lizrad/Documents/Vibrio_folder/reac_xref2.tsv" , sep='\t', low_memory=False)
df_seed = reac_xref_MNX_seed.loc[153642:170635,'XREF':'MNX_ID']
pivoted_seedd = df_seed.pivot_table(index='XREF',columns='XREF_ID',values='MNX_ID',aggfunc = lambda x: x)
#pivoted_seedd=pivoted_seed.drop(['metanetx'])
pivoted_seedd
reac_xref_MNX_seed_dict = pivoted_seedd.to_dict('index')
reac_xref_MNX_seed_dict2 = chem_xref_MNX_seed_dict['seed']
reac_xref_MNX_seed_dict2

XREF_ID,MNXR134184,MNXR134185,MNXR134186,MNXR134187,MNXR134188,MNXR134189,MNXR134190,MNXR134191,MNXR134192,MNXR134193,...,rxn14421,rxn14422,rxn14423,rxn14424,rxn14425,rxn14426,rxn14427,rxn14428,rxn14429,rxn14430
XREF,,,,,,,,,,,,,,,,,,,,,
metanetx,MNXR134184,MNXR134185,MNXR134186,MNXR134187,MNXR134188,MNXR134189,MNXR134190,MNXR134191,MNXR134192,MNXR134193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
seed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,MNXR137927,MNXR137928,MNXR137929,MNXR137930,MNXR137931,MNXR137932,MNXR137933,MNXR137934,MNXR97602,MNXR137233


{'MNXR134184': nan,
 'MNXR134185': nan,
 'MNXR134186': nan,
 'MNXR134187': nan,
 'MNXR134188': nan,
 'MNXR134189': nan,
 'MNXR134190': nan,
 'MNXR134191': nan,
 'MNXR134192': nan,
 'MNXR134193': nan,
 'MNXR134194': nan,
 'MNXR134195': nan,
 'MNXR134196': nan,
 'MNXR134197': nan,
 'MNXR134198': nan,
 'MNXR134199': nan,
 'MNXR134200': nan,
 'MNXR134201': nan,
 'MNXR134202': nan,
 'MNXR134203': nan,
 'MNXR134204': nan,
 'MNXR134205': nan,
 'MNXR134206': nan,
 'MNXR134207': nan,
 'MNXR134208': nan,
 'MNXR134209': nan,
 'MNXR134210': nan,
 'MNXR134211': nan,
 'MNXR134212': nan,
 'MNXR134213': nan,
 'MNXR134214': nan,
 'MNXR134215': nan,
 'MNXR134216': nan,
 'MNXR134217': nan,
 'MNXR134218': nan,
 'MNXR134219': nan,
 'MNXR134220': nan,
 'MNXR134221': nan,
 'MNXR134222': nan,
 'MNXR134223': nan,
 'MNXR134224': nan,
 'MNXR134225': nan,
 'MNXR134226': nan,
 'MNXR134227': nan,
 'MNXR134228': nan,
 'MNXR134229': nan,
 'MNXR134230': nan,
 'MNXR134231': nan,
 'MNXR134232': nan,
 'MNXR134233': nan,


In [53]:
not_in_reac_xref = []
in_reac_xref = []

for met in model.reactions:
    split=met.id.split("_")
    met_clean=split[0]
    print(met_clean)
    if not met_clean == '':
        
        if not met_clean in reac_xref_MNX_seed_dict2.keys():
            not_in_reac_xref.append(met)
        else:
            in_reac_xref.append(met)

print ('%i metabolites with SEED ID cannot be found in the MNX database' % (len(not_in_reac_xref)))


rxn02201
rxn00351
rxn07431
rxn00836
rxn02209
rxn00390
rxn08180
rxn05318
rxn00062
rxn06472
rxn11567
rxn00423
rxn00364
rxn05561
rxn03408
rxn02177
rxn05250
rxn00646
rxn00935
rxn01673
rxn00247
rxn05625
rxn02360
rxn02342
rxn10311
rxn05156
rxn00022
rxn00973
rxn04031
rxn02000
rxn03239
rxn03120
rxn02632
rxn11962
rxn00322
rxn03883
rxn05063
rxn05457
rxn00107
rxn04271
rxn13783
rxn10118
rxn05392
rxn12008
rxn04600
rxn04681
rxn04070
rxn00199
rxn01358
rxn10231
rxn02944
rxn01867
rxn04648
rxn10270
rxn12637
rxn03046
rxn09240
rxn01219
rxn05958
rxn00519
rxn00258
rxn00800
rxn01705
rxn09177
rxn00211
rxn00179
rxn01210
rxn00730
rxn00192
rxn01218
rxn05367
rxn03456
rxn03360
rxn02898
rxn05654
rxn01199
rxn00545
rxn00771
rxn02319
rxn08807
rxn05614
rxn00501
rxn00257
rxn10344
rxn01547
rxn10336
rxn02320
rxn03384
rxn06208
rxn01405
rxn02504
rxn04082
rxn04996
rxn00262
rxn05404
rxn02796
rxn10262
rxn10126
rxn10209
rxn08084
rxn10225
rxn03463
rxn01486
rxn00898
rxn05334
rxn10205
rxn05040
rxn05400
rxn01636
rxn05235
rxn02090
r

rxn00202
rxn05593
rxn08295
rxn02914
rxn01603
rxn00055
rxn00686
rxn00905
rxn05456
rxn02409
rxn09112
rxn00157
rxn03194
rxn06075
rxn03898
rxn00689
rxn08129
rxn01286
rxn05289
rxn01973
rxn05467
rxn00117
rxn05343
rxn01520
rxn05425
rxn03395
rxn00330
rxn05736
rxn03445
rxn01641
rxn11663
rxn00493
rxn00933
rxn08298
rxn01834
rxn00293
rxn00948
rxn05383
rxn00741
rxn11584
rxn00839
rxn03838
rxn03439
rxn05390
rxn05386
rxn10473
rxn05435
rxn05391
rxn00851
rxn05329
rxn08618
rxn08620
rxn05433
rxn05438
rxn03916
rxn00979
rxn03918
rxn05437
rxn02774
rxn03164
rxn08619
rxn03917
rxn06848
rxn02404
rxn08712
rxn08708
rxn08709
rxn05434
rxn05444
rxn05361
rxn05514
rxn05406
rxn05039
rxn05365
rxn05398
rxn03182
rxn05369
rxn05402
rxn05381
rxn08583
rxn05436
rxn05443
rxn05439
rxn08954
rxn08333
rxn05441
rxn05394
rxn05373
rxn05442
rxn09225
rxn05618
rxn05377
rxn08713
bio1
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX
EX

### SEED Ids to reaction annotation

In [63]:
for reac in model.reactions:
    if reac.id.startswith('rxn'):
        split, compartment=reac.id.split("_")
        reac.annotation['seed.compound']=split
        print(reac.annotation)
        compartment
        

{'seed.compound': 'rxn02201'}


'c0'

{'seed.compound': 'rxn00351'}


'c0'

{'seed.compound': 'rxn07431'}


...

'c0'

{'seed.compound': 'rxn05377'}


'c0'

{'seed.compound': 'rxn08713'}


'c0'

In [69]:
for reac in model.reactions:
    data=''
    if 'seed.compound' in reac.annotation:
        data=reac.annotation['seed.compound'] 
        
        for xref in reac_xref_MNX_seed_dict2.keys():
            if xref in data:
                reac.annotation['metanetx.reaction']=reac_xref_MNX_seed_dict2[xref]
                print(reac.annotation)

{'seed.compound': 'rxn02201', 'metanetx.reaction': 'MNXR97439'}
{'seed.compound': 'rxn00351', 'metanetx.reaction': 'MNXR100450'}
{'seed.compound': 'rxn07431', 'metanetx.reaction': 'MNXR101695'}
{'seed.compound': 'rxn00836', 'metanetx.reaction': 'MNXR100752'}
{'seed.compound': 'rxn02209', 'metanetx.reaction': 'MNXR95713'}
{'seed.compound': 'rxn00390', 'metanetx.reaction': 'MNXR106577'}
{'seed.compound': 'rxn08180', 'metanetx.reaction': 'MNXR96343'}
{'seed.compound': 'rxn05318', 'metanetx.reaction': 'MNXR95461'}
{'seed.compound': 'rxn00062', 'metanetx.reaction': 'MNXR96131'}
{'seed.compound': 'rxn06472', 'metanetx.reaction': 'MNXR108396'}
{'seed.compound': 'rxn11567', 'metanetx.reaction': 'MNXR108115'}
{'seed.compound': 'rxn00423', 'metanetx.reaction': 'MNXR104275'}
{'seed.compound': 'rxn00364', 'metanetx.reaction': 'MNXR97047'}
{'seed.compound': 'rxn05561', 'metanetx.reaction': 'MNXR99713'}
{'seed.compound': 'rxn03408', 'metanetx.reaction': 'MNXR105037'}
{'seed.compound': 'rxn02177', 'm

{'seed.compound': 'rxn01974', 'metanetx.reaction': 'MNXR97143'}
{'seed.compound': 'rxn01287', 'metanetx.reaction': 'MNXR107155'}
{'seed.compound': 'rxn00242', 'metanetx.reaction': 'MNXR103109'}
{'seed.compound': 'rxn12641', 'metanetx.reaction': 'MNXR123346'}
{'seed.compound': 'rxn00068', 'metanetx.reaction': 'MNXR106384'}
{'seed.compound': 'rxn00568', 'metanetx.reaction': 'MNXR102062'}
{'seed.compound': 'rxn08971', 'metanetx.reaction': 'MNXR101863'}
{'seed.compound': 'rxn05316', 'metanetx.reaction': 'MNXR100850'}
{'seed.compound': 'rxn01635', 'metanetx.reaction': 'MNXR107426'}
{'seed.compound': 'rxn04384', 'metanetx.reaction': 'MNXR95439'}
{'seed.compound': 'rxn03887', 'metanetx.reaction': 'MNXR99955'}
{'seed.compound': 'rxn00060', 'metanetx.reaction': 'MNXR100658'}
{'seed.compound': 'rxn00225', 'metanetx.reaction': 'MNXR95269'}
{'seed.compound': 'rxn05350', 'metanetx.reaction': 'MNXR109248'}
{'seed.compound': 'rxn09198', 'metanetx.reaction': 'MNXR135981'}
{'seed.compound': 'rxn08838',

{'seed.compound': 'rxn09188', 'metanetx.reaction': 'MNXR103187'}
{'seed.compound': 'rxn01422', 'metanetx.reaction': 'MNXR134296'}
{'seed.compound': 'rxn09180', 'metanetx.reaction': 'MNXR103130'}
{'seed.compound': 'rxn08822', 'metanetx.reaction': 'MNXR101167'}
{'seed.compound': 'rxn04677', 'metanetx.reaction': 'MNXR110599'}
{'seed.compound': 'rxn02782', 'metanetx.reaction': 'MNXR101764'}
{'seed.compound': 'rxn02821', 'metanetx.reaction': 'MNXR134405'}
{'seed.compound': 'rxn10208', 'metanetx.reaction': 'MNXR136205'}
{'seed.compound': 'rxn01620', 'metanetx.reaction': 'MNXR99470'}
{'seed.compound': 'rxn05345', 'metanetx.reaction': 'MNXR109082'}
{'seed.compound': 'rxn04068', 'metanetx.reaction': 'MNXR109872'}
{'seed.compound': 'rxn01522', 'metanetx.reaction': 'MNXR105225'}
{'seed.compound': 'rxn00336', 'metanetx.reaction': 'MNXR100789'}
{'seed.compound': 'rxn05422', 'metanetx.reaction': 'MNXR134919'}
{'seed.compound': 'rxn03891', 'metanetx.reaction': 'MNXR100621'}
{'seed.compound': 'rxn0146

{'seed.compound': 'rxn06022', 'metanetx.reaction': 'MNXR95411'}
{'seed.compound': 'rxn01124', 'metanetx.reaction': 'MNXR107048'}
{'seed.compound': 'rxn00285', 'metanetx.reaction': 'MNXR104635'}
{'seed.compound': 'rxn05371', 'metanetx.reaction': 'MNXR134868'}
{'seed.compound': 'rxn04794', 'metanetx.reaction': 'MNXR102135'}
{'seed.compound': 'rxn07584', 'metanetx.reaction': 'MNXR101081'}
{'seed.compound': 'rxn00182', 'metanetx.reaction': 'MNXR100085'}
{'seed.compound': 'rxn01416', 'metanetx.reaction': 'MNXR96956'}
{'seed.compound': 'rxn05567', 'metanetx.reaction': 'MNXR100018'}
{'seed.compound': 'rxn02181', 'metanetx.reaction': 'MNXR100691'}
{'seed.compound': 'rxn03935', 'metanetx.reaction': 'MNXR109746'}
{'seed.compound': 'rxn00532', 'metanetx.reaction': 'MNXR106663'}
{'seed.compound': 'rxn05172', 'metanetx.reaction': 'MNXR104732'}
{'seed.compound': 'rxn01642', 'metanetx.reaction': 'MNXR100896'}
{'seed.compound': 'rxn00132', 'metanetx.reaction': 'MNXR102037'}
{'seed.compound': 'rxn11724

{'seed.compound': 'rxn01215', 'metanetx.reaction': 'MNXR107111'}
{'seed.compound': 'rxn12224', 'metanetx.reaction': 'MNXR136820'}
{'seed.compound': 'rxn05322', 'metanetx.reaction': 'MNXR108883'}
{'seed.compound': 'rxn12848', 'metanetx.reaction': 'MNXR137158'}
{'seed.compound': 'rxn03003', 'metanetx.reaction': 'MNXR102410'}
{'seed.compound': 'rxn05151', 'metanetx.reaction': 'MNXR101265'}
{'seed.compound': 'rxn03537', 'metanetx.reaction': 'MNXR95216'}
{'seed.compound': 'rxn08804', 'metanetx.reaction': 'MNXR135848'}
{'seed.compound': 'rxn01316', 'metanetx.reaction': 'MNXR95297'}
{'seed.compound': 'rxn00966', 'metanetx.reaction': 'MNXR96711'}
{'seed.compound': 'rxn13782', 'metanetx.reaction': 'MNXR137715'}
{'seed.compound': 'rxn04270', 'metanetx.reaction': 'MNXR110224'}
{'seed.compound': 'rxn05430', 'metanetx.reaction': 'MNXR134927'}
{'seed.compound': 'rxn11838', 'metanetx.reaction': 'MNXR111862'}
{'seed.compound': 'rxn01917', 'metanetx.reaction': 'MNXR95256'}
{'seed.compound': 'rxn10236',

...

{'seed.compound': 'rxn00328', 'metanetx.reaction': 'MNXR97219'}
{'seed.compound': 'rxn02285', 'metanetx.reaction': 'MNXR105041'}
{'seed.compound': 'rxn02303', 'metanetx.reaction': 'MNXR96880'}
{'seed.compound': 'rxn03087', 'metanetx.reaction': 'MNXR104301'}
{'seed.compound': 'rxn00227', 'metanetx.reaction': 'MNXR95427'}
{'seed.compound': 'rxn03047', 'metanetx.reaction': 'MNXR95866'}
{'seed.compound': 'rxn02889', 'metanetx.reaction': 'MNXR101411'}
{'seed.compound': 'rxn05663', 'metanetx.reaction': 'MNXR104950'}
{'seed.compound': 'rxn01109', 'metanetx.reaction': 'MNXR107039'}
{'seed.compound': 'rxn08848', 'metanetx.reaction': 'MNXR135888'}
{'seed.compound': 'rxn03934', 'metanetx.reaction': 'MNXR109745'}
{'seed.compound': 'rxn07883', 'metanetx.reaction': 'MNXR111673'}
{'seed.compound': 'rxn00691', 'metanetx.reaction': 'MNXR99669'}
{'seed.compound': 'rxn08840', 'metanetx.reaction': 'MNXR135882'}
{'seed.compound': 'rxn00807', 'metanetx.reaction': 'MNXR106820'}
{'seed.compound': 'rxn05559', 

{'seed.compound': 'rxn03249', 'metanetx.reaction': 'MNXR100543'}
{'seed.compound': 'rxn04113', 'metanetx.reaction': 'MNXR100810'}
{'seed.compound': 'rxn00676', 'metanetx.reaction': 'MNXR95195'}
{'seed.compound': 'rxn05551', 'metanetx.reaction': 'MNXR97975'}
{'seed.compound': 'rxn00713', 'metanetx.reaction': 'MNXR105163'}
{'seed.compound': 'rxn06515', 'metanetx.reaction': 'MNXR108494'}
{'seed.compound': 'rxn10218', 'metanetx.reaction': 'MNXR136214'}
{'seed.compound': 'rxn01257', 'metanetx.reaction': 'MNXR95440'}
{'seed.compound': 'rxn09272', 'metanetx.reaction': 'MNXR99641'}
{'seed.compound': 'rxn02970', 'metanetx.reaction': 'MNXR134425'}
{'seed.compound': 'rxn03693', 'metanetx.reaction': 'MNXR109540'}
{'seed.compound': 'rxn03845', 'metanetx.reaction': 'MNXR109679'}
{'seed.compound': 'rxn00903', 'metanetx.reaction': 'MNXR96230'}
{'seed.compound': 'rxn00980', 'metanetx.reaction': 'MNXR102543'}
{'seed.compound': 'rxn00056', 'metanetx.reaction': 'MNXR99561'}
{'seed.compound': 'rxn00016', '

{'seed.compound': 'rxn00912', 'metanetx.reaction': 'MNXR101702'}
{'seed.compound': 'rxn05644', 'metanetx.reaction': 'MNXR104035'}
{'seed.compound': 'rxn12147', 'metanetx.reaction': 'MNXR112124'}
{'seed.compound': 'rxn08819', 'metanetx.reaction': 'MNXR135863'}
{'seed.compound': 'rxn02345', 'metanetx.reaction': 'MNXR108069'}
{'seed.compound': 'rxn06798', 'metanetx.reaction': 'MNXR115761'}
{'seed.compound': 'rxn03849', 'metanetx.reaction': 'MNXR109682'}
{'seed.compound': 'rxn03512', 'metanetx.reaction': 'MNXR102398'}
{'seed.compound': 'rxn00122', 'metanetx.reaction': 'MNXR95501'}
{'seed.compound': 'rxn09979', 'metanetx.reaction': 'MNXR96248'}
{'seed.compound': 'rxn06076', 'metanetx.reaction': 'MNXR104064'}
{'seed.compound': 'rxn00985', 'metanetx.reaction': 'MNXR106946'}
{'seed.compound': 'rxn10212', 'metanetx.reaction': 'MNXR120587'}
{'seed.compound': 'rxn01011', 'metanetx.reaction': 'MNXR100325'}
{'seed.compound': 'rxn12849', 'metanetx.reaction': 'MNXR137159'}
{'seed.compound': 'rxn10132

{'seed.compound': 'rxn05128', 'metanetx.reaction': 'MNXR100114'}
{'seed.compound': 'rxn10127', 'metanetx.reaction': 'MNXR104879'}
{'seed.compound': 'rxn10770', 'metanetx.reaction': 'MNXR96697'}
{'seed.compound': 'rxn04654', 'metanetx.reaction': 'MNXR110583'}
{'seed.compound': 'rxn10161', 'metanetx.reaction': 'MNXR102142'}
{'seed.compound': 'rxn05089', 'metanetx.reaction': 'MNXR115002'}
{'seed.compound': 'rxn05360', 'metanetx.reaction': 'MNXR134857'}
{'seed.compound': 'rxn02297', 'metanetx.reaction': 'MNXR95854'}
{'seed.compound': 'rxn07901', 'metanetx.reaction': 'MNXR111692'}
{'seed.compound': 'rxn05421', 'metanetx.reaction': 'MNXR134918'}
{'seed.compound': 'rxn01799', 'metanetx.reaction': 'MNXR97818'}
{'seed.compound': 'rxn00897', 'metanetx.reaction': 'MNXR95247'}
{'seed.compound': 'rxn01485', 'metanetx.reaction': 'MNXR102526'}
{'seed.compound': 'rxn09104', 'metanetx.reaction': 'MNXR102563'}
{'seed.compound': 'rxn05206', 'metanetx.reaction': 'MNXR100950'}
{'seed.compound': 'rxn02190',

{'seed.compound': 'rxn00741', 'metanetx.reaction': 'MNXR94796'}
{'seed.compound': 'rxn11584', 'metanetx.reaction': 'MNXR108565'}
{'seed.compound': 'rxn00839', 'metanetx.reaction': 'MNXR101940'}
{'seed.compound': 'rxn03838', 'metanetx.reaction': 'MNXR101747'}
{'seed.compound': 'rxn03439', 'metanetx.reaction': 'MNXR101684'}
{'seed.compound': 'rxn05390', 'metanetx.reaction': 'MNXR134887'}
{'seed.compound': 'rxn05386', 'metanetx.reaction': 'MNXR134883'}
{'seed.compound': 'rxn10473', 'metanetx.reaction': 'MNXR96797'}
{'seed.compound': 'rxn05435', 'metanetx.reaction': 'MNXR134932'}
{'seed.compound': 'rxn05391', 'metanetx.reaction': 'MNXR134888'}
{'seed.compound': 'rxn00851', 'metanetx.reaction': 'MNXR95678'}
{'seed.compound': 'rxn05329', 'metanetx.reaction': 'MNXR94886'}
{'seed.compound': 'rxn08618', 'metanetx.reaction': 'MNXR135833'}
{'seed.compound': 'rxn08620', 'metanetx.reaction': 'MNXR100228'}
{'seed.compound': 'rxn05433', 'metanetx.reaction': 'MNXR134930'}
{'seed.compound': 'rxn05438',

In [8]:
chem_xref['#XREF'] = chem_xref['#XREF'].str.replace(':','\t')

^[0-9A-Za-z]*:[0-9A-Za-z]*
    (^[0-9A-Za-z]*):([0-9A-Za-z]*)
        $1 \t $2
        
    chem_xref



SyntaxError: invalid syntax (<ipython-input-8-50d8dc2a7371>, line 3)